In [12]:
pip install openpyxl

Note: you may need to restart the kernel to use updated packages.


In [13]:
pip install pandas

  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
   ---------------------- ----------------- 6.3/11.0 MB 32.2 MB/s eta 0:00:01
   ---------------------------------------  10.7/11.0 MB 32.0 MB/s eta 0:00:01
   ---------------------------------------- 11.0/11.0 MB 25.4 MB/s  0:00:00
   ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
   ---------------------- ----------------- 7.3/12.8 MB 37.8 MB/s eta 0:00:01
   ---------------------------------------  12.6/12.8 MB 35.9 MB/s eta 0:00:01
   ---------------------------------------- 12.8/12.8 MB 28.7 MB/s  0:00:00
Using cached tzdata-2025.2-py2.py3-none-any.whl (347 kB)

   ---------------------------------------- 0/4 [pytz]
   ---------------------------------------- 0/4 [pytz]
   ---------------------------------------- 0/4 [pytz]
   -----------------------------------

In [2]:
import pandas as pd

In [ ]:
from pathlib import Path
# from caas_jupyter_tools import display_dataframe_to_user  # <-- 1. BORRÁ O COMENTÁ ESTA LÍNEA


# 1) Cargar el archivo una sola vez
xls = pd.ExcelFile('Totalizadores Planta de Cerveza 2023_2024.xlsx')

# 2) Crear un dict con un DataFrame por hoja
dfs = {}
resumen = []


for hoja in xls.sheet_names:
    df = pd.read_excel(xls, sheet_name=hoja)
    dfs[hoja] = df
    resumen.append({
        "hoja": hoja,
        "filas": len(df),
        "columnas": df.shape[1],
        "nombres_columnas": ", ".join(map(str, df.columns.tolist()))
    })

# 3) Mostrar un resumen amigable
resumen_df = pd.DataFrame(resumen)

# --- 2. REEMPLAZÁ LA LÍNEA ORIGINAL POR ESTAS DOS ---
print("--- Resumen de hojas y columnas ---")
print(resumen_df)
# ---------------------------------------------------

# Nota: Los DataFrames quedan disponibles en el dict dfs (ej: dfs["NombreDeLaHoja"])

--- Resumen de hojas y columnas ---
                         hoja  filas  columnas  \
0             Consolidado KPI  12010       125   
1                       Metas     48        57   
2      Consolidado Produccion  12011        19   
3    Totalizadores Produccion  12009        41   
4              Consolidado EE  12011        24   
5       Totalizadores Energia  12009        54   
6            Consolidado Agua  12011        24   
7          Totalizadores Agua  12009        44   
8        Consolidado GasVapor  12011        20   
9   Totalizadores Gas y Vapor  12009        24   
10           Consolidado Aire  12011        14   
11         Totalizadores Aire  12009        12   
12          Totalizadores CO2  12009         9   
13    Totalizadores Efluentes  12009         9   
14       Totalizadores Glicol  12009         8   
15            Seguimiento Dia  12009         4   
16                   Auxiliar  12011        38   
17          Kw Frio  Hl Mosto  12372        15   

             

In [8]:
# El diccionario 'dfs' ya está cargado de los pasos anteriores.
# No es necesario volver a cargar los archivos.

# --- 1. ELEGÍ LA HOJA QUE QUERÉS VER ---
nombre_hoja_para_ver = 'Consolidado KPI'
# (Podés cambiar 'Consolidado Produccion' por cualquier otro nombre de hoja)


# --- 2. Verificamos si la hoja existe y la imprimimos ---
if nombre_hoja_para_ver in dfs:
    
    print(f"===============================================================")
    print(f"  Mostrando la hoja: {nombre_hoja_para_ver}")
    print(f"===============================================================")
    
    # Configurar pandas para que muestre todas las columnas
    pd.set_option('display.max_columns', None) 
    pd.set_option('display.width', 1000)

    print("\n--- PRIMERAS 5 FILAS (.head()) ---")
    # Usamos .to_string() para asegurar que no corte la salida
    print(dfs[nombre_hoja_para_ver].head(5).to_string())
    
    print("\n\n--- ÚLTIMAS 5 FILAS (.tail()) ---")
    print(dfs[nombre_hoja_para_ver].tail(5).to_string())

else:
    print(f"Error: No se encontró la hoja '{nombre_hoja_para_ver}' en el diccionario dfs.")
    print("Las hojas disponibles son:")
    print(list(dfs.keys()))

  Mostrando la hoja: Consolidado KPI

--- PRIMERAS 5 FILAS (.head()) ---
         DIA      HORA  EE Planta / Hl  EE Elaboracion / Hl  EE Bodega / Hl  EE Cocina / Hl  EE Envasado / Hl  EE Linea 2 / Hl  EE Linea 3 / Hl  EE Linea 4 / Hl  EE Linea 5 / Hl  EE Servicios / Hl  EE Sala Maq / Hl  EE Frio / Hl  EE Aire / Hl  EE CO2 / Hl  EE Caldera / Hl  EE Eflu / Hl  EE Agua / Hl  EE Resto Serv / Hl  EE Resto Planta / Hl  Unnamed: 21  Unnamed: 22  Agua Planta / Hl  Agua Elab / Hl  Agua Bodega / Hl  Agua Cocina / Hl  Agua Envas / Hl  Agua Linea 2/Hl  Agua Linea 3/Hl  Agua Linea 4/Hl  Agua Linea 5/Hl  Agua Servicios/Hl  Agua Planta de Agua/Hl  Produccion Agua / Hl  Unnamed: 35  ET Planta / Hl  ET Elab/Hl  ET Bodega/Hl  ET Cocina/Hl  ET Envasado/Hl  ET Linea 2/Hl  ET Linea 3/Hl  ET Linea 4/Hl  ET Linea 5/Hl  ET Servicios / Hl  Unnamed: 46  Aire Planta / Hl  Aire Elaboracion / Hl  Aire Cocina / Hl  Aire Bodega / Hl  Aire Envasado / Hl  Aire L2 / Hl  Aire L3 / Hl  Aire L4 / Hl  Aire L5 / Hl  Aire Se

In [9]:
import pandas as pd
import numpy as np

print("\n--- Análisis de Cobertura de Datos ---")

# Lista para guardar los resultados de cada hoja
resultados_analisis = []

# Función para formatear las listas de días y que no saturen la salida
def format_lista_dias(lista):
    if not lista:
        return "Ninguno"
    if len(lista) > 3:
        # Mostrar los primeros 3 y el total
        primeros_tres = ', '.join(map(str, lista[:3]))
        return f"{len(lista)} días (Ej: {primeros_tres}, ...)"
    else:
        return ', '.join(map(str, lista))

# Iterar sobre el dict de DataFrames que ya creaste
for hoja, df in dfs.items():
    
    # 1. Verificar si la hoja tiene las columnas 'DIA' y 'HORA'
    if 'DIA' not in df.columns or 'HORA' not in df.columns:
        
        # Intentar analizar hojas solo con fecha (como 'Metas')
        col_fecha_alt = next((col for col in ['Mes / Año', 'Dia'] if col in df.columns), None)
        if col_fecha_alt:
            try:
                fechas_alt = pd.to_datetime(df[col_fecha_alt], errors='coerce').dropna()
                if not fechas_alt.empty:
                    resultados_analisis.append({
                        "hoja": hoja,
                        "primer_dia": fechas_alt.min().date(),
                        "ultimo_dia": fechas_alt.max().date(),
                        "dias_sin_23_59": "N/A (Hoja no horaria)",
                        "dias_con_horas_faltantes": "N/A (Hoja no horaria)"
                    })
            except Exception:
                pass # Omitir si falla
        continue # Saltar esta hoja si no tiene DIA y HORA

    try:
        # 2. Preparar los datos
        df_proc = df.copy()
        
        # Convertir 'DIA' a datetime (solo la fecha)
        # errors='coerce' convierte fechas inválidas en NaT (Not a Time)
        df_proc['DIA_fecha'] = pd.to_datetime(df_proc['DIA'], errors='coerce').dt.date
        
        # Convertir 'HORA' a string para buscar '23:59' de forma segura
        df_proc['HORA_str'] = df_proc['HORA'].astype(str)
        
        # Eliminar filas donde la fecha no se pudo parsear
        df_proc = df_proc.dropna(subset=['DIA_fecha'])
        
        if df_proc.empty:
            continue # Saltar hoja si no hay datos de fecha válidos

        # 3. (Goal 2) Primer y último día
        primer_dia = df_proc['DIA_fecha'].min()
        ultimo_dia = df_proc['DIA_fecha'].max()
        
        # Días únicos que SÍ tienen el registro '23:59'
        # Usamos .str.contains() para capturar '23:59:00' o '23:59'
        dias_con_23_59 = df_proc[df_proc['HORA_str'].str.contains('23:59')]['DIA_fecha'].unique()
        
        # Todos los días únicos en el dataset de esta hoja
        todos_los_dias = df_proc['DIA_fecha'].unique()
        
        # 4. (Goal 1) Días que NO tienen 23:59 (Diferencia de conjuntos)
        dias_sin_23_59_lista = sorted(list(set(todos_los_dias) - set(dias_con_23_59)))

        # 5. (Goal 3) Días con horas faltantes
        # Contamos cuántos registros (horas) hay por cada día
        registros_por_dia = df_proc.groupby('DIA_fecha').size()
        
        # Un día debe tener al menos 24 registros (00:00 a 23:00).
        # Si tiene menos de 24, le faltan horas.
        dias_con_horas_faltantes_sr = registros_por_dia[registros_por_dia < 24]
        dias_con_horas_faltantes_lista = sorted(list(dias_con_horas_faltantes_sr.index))

        # 6. Guardar resultados
        resultados_analisis.append({
            "hoja": hoja,
            "primer_dia": primer_dia,
            "ultimo_dia": ultimo_dia,
            "dias_sin_23_59": format_lista_dias(dias_sin_23_59_lista),
            "dias_con_horas_faltantes": format_lista_dias(dias_con_horas_faltantes_lista)
        })

    except Exception as e:
        # Registrar error si algo falla en una hoja específica
        resultados_analisis.append({
            "hoja": hoja,
            "primer_dia": f"Error: {e}",
            "ultimo_dia": f"Error: {e}",
            "dias_sin_23_59": "Error",
            "dias_con_horas_faltantes": "Error"
        })

# 7. Mostrar el reporte final
if resultados_analisis:
    reporte_df = pd.DataFrame(resultados_analisis).set_index('hoja')
    
    # Configurar pandas para mostrar bien el resultado
    pd.set_option('display.max_colwidth', 200) # Para que no corte las listas
    pd.set_option('display.width', 1000)       # Para que use más ancho de pantalla
    
    print(reporte_df)
else:
    print("No se encontraron hojas con las columnas 'DIA' y 'HORA' para analizar.")


--- Análisis de Cobertura de Datos ---
                           primer_dia  ultimo_dia                                        dias_sin_23_59                              dias_con_horas_faltantes
hoja                                                                                                                                                         
Consolidado KPI            2023-01-01  2024-10-26  4 días (Ej: 2023-02-28, 2023-04-13, 2023-04-19, ...)  5 días (Ej: 2023-01-17, 2023-04-13, 2023-04-19, ...)
Metas                      2021-01-01  2024-12-01                                 N/A (Hoja no horaria)                                 N/A (Hoja no horaria)
Consolidado Produccion     2023-01-01  2024-10-26  4 días (Ej: 2023-02-28, 2023-04-13, 2023-04-19, ...)  5 días (Ej: 2023-01-17, 2023-04-13, 2023-04-19, ...)
Totalizadores Produccion   2023-01-01  2024-10-26  4 días (Ej: 2023-02-28, 2023-04-13, 2023-04-19, ...)  5 días (Ej: 2023-01-17, 2023-04-13, 2023-04-19, ...)
Consolidado 

In [10]:
dias_sin_23_59_lista

[datetime.date(2023, 2, 28),
 datetime.date(2023, 4, 13),
 datetime.date(2023, 4, 19),
 datetime.date(2024, 10, 26)]

In [11]:
import pandas as pd
import numpy as np

# Asegúrate de que el diccionario 'dfs' exista y esté cargado
if 'dfs' not in globals() or not isinstance(dfs, dict) or not dfs:
    print("Error: El diccionario 'dfs' no se encontró en memoria o está vacío.")
    # Si 'dfs' no existe, deberías correr primero el código que carga los CSVs.
else:
    print("--- Iniciando Análisis de Días Faltantes (Gaps) ---")

    # Lista para guardar los resultados
    resultados_dias_faltantes = []

    # Función para formatear las listas de días
    def format_lista_dias(lista):
        if not lista:
            return "Ninguno"
        # Convertir fechas a strings
        lista_str = [str(d) for d in lista]
        if len(lista_str) > 3:
            primeros_tres = ', '.join(lista_str[:3])
            return f"{len(lista_str)} días (Ej: {primeros_tres}, ...)"
        else:
            return ', '.join(lista_str)

    # Iterar sobre el dict de DataFrames
    for hoja in sorted(dfs.keys()):
        df = dfs[hoja]
        
        # --- 1. Identificar columnas de fecha (lógica ya validada) ---
        date_col = None
        if 'DIA.1' in df.columns and 'HORA.1' in df.columns:
            date_col = 'DIA.1'
        elif 'DIA' in df.columns and 'HORA' in df.columns:
            date_col = 'DIA'
        elif 'Dia' in df.columns and 'Hora' in df.columns:
            date_col = 'Dia'
        elif 'Mes / Año' in df.columns:
            # Lógica para hojas mensuales como 'Metas'
            try:
                fechas_alt = pd.to_datetime(df['Mes / Año'], errors='coerce').dropna().dt.date
                if not fechas_alt.empty:
                    primer_dia_alt = fechas_alt.min()
                    ultimo_dia_alt = fechas_alt.max()
                    
                    # Para 'Metas', chequeamos meses faltantes
                    ideal_range_mes = pd.date_range(start=primer_dia_alt, end=ultimo_dia_alt, freq='MS') # MS = Month Start
                    ideal_meses_set = set(ideal_range_mes.date)
                    presentes_meses_set = set(fechas_alt)
                    
                    meses_faltantes = sorted(list(ideal_meses_set - presentes_meses_set))
                    
                    resultados_dias_faltantes.append({
                        "hoja": hoja,
                        "primer_dia": primer_dia_alt,
                        "ultimo_dia": ultimo_dia_alt,
                        "dias_faltantes": f"N/A (Mensual) - {format_lista_dias(meses_faltantes)}"
                    })
            except Exception:
                pass
            continue # Saltar al siguiente loop
        
        # Si no encontramos columnas, saltar
        if date_col is None:
            continue

        # --- 2. Procesar datos ---
        try:
            df_proc = df.copy()
            
            # Convertir col de fecha a datetime y extraer solo la fecha
            df_proc['DIA_fecha'] = pd.to_datetime(df_proc[date_col], errors='coerce').dt.date
            
            # Limpiar filas donde la fecha no se pudo parsear
            df_proc = df_proc.dropna(subset=['DIA_fecha'])
            
            if df_proc.empty:
                continue # Saltar hoja si no hay datos de fecha válidos

            # --- 3. Análisis de Primer/Último Día ---
            primer_dia = df_proc['DIA_fecha'].min()
            ultimo_dia = df_proc['DIA_fecha'].max()
            
            # --- 4. (NUEVO) Análisis de Días Faltantes ---
            
            # Obtener el set de días únicos PRESENTES en los datos
            dias_presentes = set(df_proc['DIA_fecha'].unique())
            
            # Crear el set de días IDEAL (todos los días desde el inicio al fin)
            # pd.date_range es inclusivo
            ideal_range = pd.date_range(start=primer_dia, end=ultimo_dia, freq='D')
            
            # Convertir el rango ideal a un set de objetos 'date' para comparar
            ideal_dias_set = set(ideal_range.date)
            
            # Calcular la diferencia: Días ideales MENOS Días presentes
            dias_faltantes_lista = sorted(list(ideal_dias_set - dias_presentes))

            # --- 5. Guardar resultados ---
            resultados_dias_faltantes.append({
                "hoja": hoja,
                "primer_dia": primer_dia,
                "ultimo_dia": ultimo_dia,
                "dias_faltantes": format_lista_dias(dias_faltantes_lista)
            })

        except Exception as e:
            resultados_dias_faltantes.append({
                "hoja": hoja,
                "primer_dia": f"Error: {e}",
                "ultimo_dia": "Error",
                "dias_faltantes": "Error"
            })

    # --- 6. Mostrar el reporte final ---
    if resultados_dias_faltantes:
        reporte_df = pd.DataFrame(resultados_dias_faltantes).set_index('hoja')
        
        # Reordenar para que coincida con el orden de carga (alfabético)
        reporte_df = reporte_df.reindex(sorted(dfs.keys()))
        
        pd.set_option('display.max_colwidth', 200)
        pd.set_option('display.width', 1000)
        
        print("\n--- Reporte de Días Faltantes (Gaps) ---")
        print(reporte_df.to_string())
    else:
        print("No se generaron resultados de análisis.")

    print("\n--- Fin del Análisis ---")

--- Iniciando Análisis de Días Faltantes (Gaps) ---

--- Reporte de Días Faltantes (Gaps) ---
                           primer_dia  ultimo_dia                                          dias_faltantes
hoja                                                                                                     
Auxiliar                   2023-01-01  2024-04-26                                                 Ninguno
Consolidado Agua           2023-01-01  2024-10-26  188 días (Ej: 2023-03-31, 2023-05-31, 2023-10-31, ...)
Consolidado Aire           2023-01-01  2024-10-26  188 días (Ej: 2023-03-31, 2023-05-31, 2023-10-31, ...)
Consolidado EE             2023-01-01  2024-10-26  188 días (Ej: 2023-03-31, 2023-05-31, 2023-10-31, ...)
Consolidado GasVapor       2023-01-01  2024-10-26  188 días (Ej: 2023-03-31, 2023-05-31, 2023-10-31, ...)
Consolidado KPI            2023-01-01  2024-10-26  188 días (Ej: 2023-03-31, 2023-05-31, 2023-10-31, ...)
Consolidado Produccion     2023-01-01  2024-10-26  188 día

In [12]:
dias_faltantes_lista

[datetime.date(2023, 3, 31),
 datetime.date(2023, 5, 31),
 datetime.date(2023, 10, 31),
 datetime.date(2023, 12, 31),
 datetime.date(2024, 1, 1),
 datetime.date(2024, 1, 2),
 datetime.date(2024, 1, 3),
 datetime.date(2024, 1, 4),
 datetime.date(2024, 1, 5),
 datetime.date(2024, 1, 6),
 datetime.date(2024, 1, 7),
 datetime.date(2024, 1, 8),
 datetime.date(2024, 1, 9),
 datetime.date(2024, 1, 10),
 datetime.date(2024, 1, 11),
 datetime.date(2024, 1, 12),
 datetime.date(2024, 1, 13),
 datetime.date(2024, 1, 14),
 datetime.date(2024, 1, 15),
 datetime.date(2024, 1, 16),
 datetime.date(2024, 1, 17),
 datetime.date(2024, 1, 18),
 datetime.date(2024, 1, 19),
 datetime.date(2024, 1, 20),
 datetime.date(2024, 1, 21),
 datetime.date(2024, 1, 22),
 datetime.date(2024, 1, 23),
 datetime.date(2024, 1, 24),
 datetime.date(2024, 1, 25),
 datetime.date(2024, 1, 26),
 datetime.date(2024, 1, 27),
 datetime.date(2024, 1, 28),
 datetime.date(2024, 1, 29),
 datetime.date(2024, 1, 30),
 datetime.date(2024, 